<a href="https://colab.research.google.com/github/Riyaa-Bajpai/ai_image_generation/blob/main/ai_img_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install diffusers transformers accelerate safetensors Pillow
!pip install streamlit
!pip install pyngrok

Looking in indexes: https://download.pytorch.org/whl/cu118
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 100.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 145.2 MB/s eta 0:00:00


In [2]:
from pyngrok import ngrok
import getpass

NGROK_TOKEN = getpass.getpass("Enter your ngrok token: ")
ngrok.set_auth_token(NGROK_TOKEN)

print("Ngrok token set successfully!")

Enter your ngrok token: ··········
Ngrok token set successfully!


In [6]:
%%writefile app.py
import os
import json
from datetime import datetime

import torch
from diffusers import StableDiffusionPipeline
from PIL import Image, ImageDraw, ImageFont
import streamlit as st


@st.cache_resource
def load_pipeline():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    dtype = torch.float16 if device == "cuda" else torch.float32

    pipe = StableDiffusionPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        torch_dtype=dtype,
        safety_checker=None
    )
    if device == "cuda":
        pipe = pipe.to(device)
        pipe.enable_attention_slicing()

    return pipe, device


def add_watermark(img, text="AI-Generated by Riya Bajpai"):
    if img.mode != "RGBA":
        img = img.convert("RGBA")

    width, height = img.size

    layer = Image.new("RGBA", img.size, (0,0,0,0))
    draw = ImageDraw.Draw(layer)
    font = ImageFont.load_default()

    try:
        bbox = draw.textbbox((0,0), text, font=font)
        w = bbox[2] - bbox[0]
        h = bbox[3] - bbox[1]
    except:
        w, h = draw.textsize(text, font)

    x = width - w - 30
    y = height - h - 30

    draw.rectangle([(x-10, y-5), (x+w+10, y+h+5)], fill=(0,0,0,100))
    draw.text((x,y), text, font=font, fill=(255,255,255,255))

    return Image.alpha_composite(img, layer).convert("RGB")


def save_metadata(folder, prompt, negative_prompt, num_images, steps, guidance):
    metadata = {
        "prompt": prompt,
        "negative_prompt": negative_prompt,
        "num_images": num_images,
        "steps": steps,
        "guidance": guidance,
        "timestamp": folder.split("_")[-1],
        "files": [f"img_{i+1}.png" for i in range(num_images)]
    }
    with open(os.path.join(folder, "metadata.json"), "w") as f:
        json.dump(metadata, f, indent=4)


def generate_images(pipe, prompt, num_images, steps, guidance, negative_prompt):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    folder = f"outputs_{timestamp}"
    os.makedirs(folder, exist_ok=True)

    images = []
    paths = []

    for i in range(num_images):
        result = pipe(prompt, negative_prompt=negative_prompt, num_inference_steps=steps, guidance_scale=guidance)
        img = add_watermark(result.images[0])
        path = f"{folder}/img_{i+1}.png"
        img.save(path)
        images.append(img)
        paths.append(path)

    save_metadata(folder, prompt, negative_prompt, num_images, steps, guidance)
    return images, folder, paths


def main():
    st.title("✨ AI Image Generator")

    pipe, device = load_pipeline()
    st.caption(f"Running on: **{device.upper()}**")

    with st.sidebar:
        st.header("⚙️ Settings")
        num_images = st.slider("Number of Images", 1, 4, 2)

        steps = st.slider("Steps\n(How long AI thinks)",10, 60, 30)

        guidance = st.slider("Guidance\n(How strongly AI follows prompt)",1.0, 15.0, 7.5)

        negative_prompt = st.text_area(
        "Negative Prompt(What not to generate)",
        "blurry, distorted, watermark, text, low quality")

    prompt = st.text_area("📝 Enter your prompt", "a magical library, floating books, golden atmosphere")

    if st.button("🚀 Generate"):
        with st.spinner("Generating..."):
            images, folder, paths = generate_images(pipe, prompt, num_images, steps, guidance, negative_prompt)

        st.success(f"Saved in: {folder}")

        cols = st.columns(2)
        for i, img in enumerate(images):
            with cols[i % 2]:
                st.image(img, caption=f"Image {i+1}")
                with open(paths[i], "rb") as f:
                    st.download_button(f"Download {i+1}", f, file_name=paths[i], mime="image/png")


if __name__ == "__main__":
    main()


Overwriting app.py


In [7]:
!streamlit run app.py &>/dev/null&


In [8]:
public_url = ngrok.connect(8501)
public_url


<NgrokTunnel: "https://0d8188e51764.ngrok-free.app" -> "http://localhost:8501">